# Super-structure scenario modelling in brightway

This notebook shows how to use the bw_scenarios module, using the example of a bike.

We start by importing an SDF file (.xslx or .csv).

## Project setup

In [ ]:
import bw2data as bd
import bw2io as bi

In [ ]:
PROJECT_NAME = "bc24-hackathon-bike-project"
BIO_DB_NAME = "ecoinvent-3.10-biosphere"

In [ ]:
# this is the current preferred way to install a biosphere database w/bw25
# it will create biosphere named "ecoinvent-3.10-biosphere"
# and add the associated LCIA methods
bi.remote.install_project(
    "ecoinvent-3.10-biosphere", PROJECT_NAME, overwrite_existing=True
)

In [ ]:
bd.projects.set_current(PROJECT_NAME)

In [ ]:
bd.databases

In [ ]:
xl_importer = bi.importers.ExcelImporter("bike_production_example.xlsx")

In [ ]:
# If there are unlinked exchanges because of biosphere name change,
# use the right biosphere name

In [ ]:
for data_item in xl_importer.data:
    for exchange in data_item["exchanges"]:
        if exchange["type"] == "biosphere" and exchange["database"] == "biosphere3":
            print(f"Update bio db name for {exchange}")
            exchange["database"] = BIO_DB_NAME

In [ ]:
xl_importer.apply_strategies()

In [ ]:
xl_importer.statistics()

In [ ]:
if xl_importer.statistics(print_stats=False)[3] == 0:
    xl_importer.write_database()

In [ ]:
bike_db = bd.Database("bike_production_example")

In [ ]:
for a in bike_db:
    print(f"{a} -> {a['code']}")

## Extract the data from the SDF file

In [ ]:
from pathlib import Path

from bw_scenarios.importer import SDFImporter

file_path = Path("sdf.xlsx")

# store the scenario data in a variable
scenarios = SDFImporter.from_excel(file_path)

In the .sdf-file it is defined how the activities are changing within the distinct scenarios.

The file used for this example is specified above.

In the graphic below, the direction of change per scenario and exchange is visualized.

![scenarios_visualized.png](scenarios_visualized.png)